In [1]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pymupdf

   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.0 MB 4.2 MB/s eta 0:00:04
   - -------------------------------------- 0.5/16.0 MB 4.2 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/16.0 MB 2.7 MB/s eta 0:00:06
   --- ------------------------------------ 1.3/16.0 MB 2.7 MB/s eta 0:00:06
   ------ --------------------------------- 2.6/16.0 MB 2.6 MB/s eta 0:00:06
   -------- ------------------------------- 3.4/16.0 MB 2.9 MB/s eta 0:00:05
   ---------- ----------------------------- 4.2/16.0 MB 3.0 MB/s eta 0:00:04
   ----------- ---------------------------- 4.7/16.0 MB 3.0 MB/s eta 0:00:04
   ------------- -------------------------- 5.2/16.0 MB 2.9 MB/s eta 0:00:04
   ------------- -------------------------- 5.5/16.0 MB 2.8 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/16.0 MB 2.6 MB/s eta 0:00:05
   --------------- ------------------------ 6.0/16.0 MB 2.4 MB/s eta 0:00:05
   ---

The main code

In [ ]:
import cv2
import pytesseract
import fitz  # PyMuPDF
import os

# Ask the user to enter the path to the PDF file
scanned_pdf = input("Please enter the path of the PDF file/image path : ")

# Check if the file exists
if not os.path.isfile(scanned_pdf):
    print("The specified file does not exist. Please check the path and try again.")
else:
    # Open the PDF with fitz (PyMuPDF)
    doc = fitz.open(scanned_pdf)

    # Create a directory to save images
    output_dir = "pdf_images"
    os.makedirs(output_dir, exist_ok=True)

    # Convert each page to an image and save
    for page_number in range(len(doc)):
        page = doc[page_number]
        pix = page.get_pixmap(dpi=300)  # Increase DPI for better quality
        image_path = os.path.join(output_dir, f'page_{page_number}.png')
        pix.save(image_path)
        print(f'Saved {image_path}')

        # Read the image using OpenCV
        image = cv2.imread(image_path)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur to reduce noise
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

        # Apply adaptive thresholding to create a binary image
        binary_image = cv2.adaptiveThreshold(blurred_image, 255,
                                             cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                             cv2.THRESH_BINARY_INV, 11, 2)

        # Apply dilation and erosion to connect broken letters and reduce noise
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        dilated_image = cv2.dilate(binary_image, kernel, iterations=1)
        eroded_image = cv2.erode(dilated_image, kernel, iterations=1)

        # Use pytesseract to extract text from the image with configuration
        custom_config = r'--oem 3 --psm 6'  # Consider using --psm 3 for single block of text or --psm 1 for automatic detection
        ocrd_text = pytesseract.image_to_string(eroded_image, config=custom_config)

        print(f"Text extracted from page {page_number} using OpenCV and Tesseract:")
        print(ocrd_text)

        # Save the extracted text to a text file
        output_text_file = "pdf_text.txt"
        with open(output_text_file, 'a', encoding='utf-8') as f:  # Append mode
            f.write(f"Text extracted from page {page_number}:\n")
            f.write(ocrd_text + "\n\n")

    print(f"Extracted text has been saved to {output_text_file}.")
